In [ ]:
!pip install wandb

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
import wandb
from wandb.keras import (
   WandbMetricsLogger,
   WandbModelCheckpoint,
)
import matplotlib.pyplot as plt
import numpy as np
import cv2

### wandb setup

In [ ]:
run = wandb.init(project="7-7-Face-Recognition")

In [ ]:
config = wandb.config

In [ ]:
wandb_callbacks = [
   WandbMetricsLogger(log_freq=5),
   WandbModelCheckpoint("models"),
]

### Preparing Dataset

In [ ]:
dataset_path = "datasets/7-7"
width = height = 224
batch_size = 32

data_generator = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    brightness_range=(0.8, 1.2),
    fill_mode='reflect',
    validation_split=0.1)

train_data = data_generator.flow_from_directory(
    dataset_path,
    target_size=(width, height),
    class_mode='categorical', 
    batch_size=batch_size,
    subset='training',
    shuffle=True
)

validation_data = data_generator.flow_from_directory(
    dataset_path,
    target_size=(width, height),
    class_mode='categorical', 
    batch_size=batch_size,
    subset='validation',
    shuffle=False
)

In [ ]:
train_images = next(train_data)[0]

for i in range(16):
    plt.subplot(4,4,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i], cmap=plt.cm.binary)

### Create Model

Option 1

In [ ]:
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(width, height, 3),
    include_top=False,
    weights="imagenet",
    pooling='avg'
)

In [ ]:
for layer in base_model.layers[:-4]:
    layer.trainable = False

In [ ]:
base_model.summary()

In [ ]:
model = Sequential([
    base_model,
    layers.Dropout(0.5),
    layers.Dense(train_data.num_classes, activation='softmax')
])

Option 2

In [ ]:
model = tf.keras.models.load_model('saved_model')

In [ ]:
model.summary()

### Train Model

In [ ]:
model.compile(
    loss=tf.keras.losses.categorical_crossentropy,
    optimizer=tf.keras.optimizers.RMSprop(lr=1e-4),
    # optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    # optimizer=tf.keras.optimizers.Adadelta(learning_rate=0.0001),
    metrics='accuracy'
)

In [ ]:
history = model.fit(
    train_data,
    validation_data=validation_data,
    epochs=32,
    callbacks=wandb_callbacks
)

In [ ]:
plt.plot(history.history['loss'], label="Train")
plt.plot(history.history['val_loss'], label="Validation")
plt.legend(loc='best')
plt.xlabel('Epochs', fontsize=16)
plt.ylabel('Loss', fontsize=16)
plt.grid()
plt.show()

plt.plot(history.history['accuracy'], label="Train")
plt.plot(history.history['val_accuracy'], label="Validation")
plt.legend(loc='best')
plt.xlabel('Epochs', fontsize=16)
plt.ylabel('Accuracy', fontsize=16)
plt.grid()
plt.show()

### Test Model

In [ ]:
import urllib.request

image_url = 'https://s.france24.com/media/display/a89f55a2-abff-11ea-b263-005056bff430/w:1280/p:16x9/2020-06-10T224534Z_26952549_RC2K6H9AXPGB_RTRMADP_3_MINNEAPOLIS-POLICE-TRUMP.webp'
urllib.request.urlretrieve(image_url, 'person.jpg')

In [ ]:
img = cv2.imread("person.jpg")
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img = cv2.resize(img, (224, 224))
img = img / 255.0
img = img.reshape(1, 224, 224, 3)

output = model.predict(img)
print(output)
print('probability', np.max(output))
print('class index', np.argmax(output))

labels = list(train_data.class_indices.keys())
print('class label', labels[np.argmax(output)])

In [ ]:
model.save('weights/7-7-face-recognition-model.h5')